#checking relation with title on X_train

In [1]:
import pandas as pd
import numpy as np
import pickle

df=pd.read_csv('train_new.csv',sep=",")
df=df[['id','title','article','tags']]

In [2]:
#inserting each tag to a seperate list
def tag(row):
    s=row['tags']
    return s.split('|')
df['tags'] = df.apply (lambda row: tag(row),axis=1)

In [3]:
df.head()

,id,title,article,tags
0,HE-d192012,incorrect behavior javascript drag drop scaled...,I have a Drag&amp;Drop function in my applicat...,"[javascript, drag-and-drop, scale]"
1,HE-e8d20ef,create nsdata 4 bytes big,I want to keep a GKSession alive with a schedu...,"[objective-c, ios, nsdata]"
2,HE-dbc6003,entity framework winforms,"Windows 7 64bit, VS 2012, SQL 2012 localdb, EF...","[winforms, entity-framework, ef-code-first, lo..."
3,HE-cbc73dc,facebook open graph tags scraped,I have thoroughly researched and tried EVERYTH...,"[wordpress, facebook, facebook-graph-api, face..."
4,HE-f57fb4d,changing tweenmax speed effects,I am pretty new to GSAP. The following works g...,"[javascript, greensock, tweenmax, gsap]"


In [4]:
df_t=pd.read_csv('test_new.csv',sep=",")
df_test1=df_t['title']

df_test_1=df_test1[:50000]
df_test_2=df_test1[50000:100000]
df_test_3=df_test1[100000:150000]
df_test_4=df_test1[150000:194323]

print(len(df_test_1))
print(len(df_test_2))
print(len(df_test_3))
print(len(df_test_4))

50000
50000
50000
44323


In [5]:
global ans
ans=[]
print(ans)
print(len(ans))

[]
0


In [6]:
df=df[['title','tags']]
#df1=df[0:30000]
#df1=df[30000:70000]
#df1=df[100000:150000]
#df1=df[150000:200000]
#df1=df[200000:250000]
#df1=df[250000:300000]
#df1=df[360000:420000]
df1=df[420000:451346]
df1.head()
print(ans)

[]


In [7]:
def train_classifier(X_train, y_train, c = 1.0):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    # Create and fit LogisticRegression wrapped into OneVsRestClassifier.
    
    mlb_clf = OneVsRestClassifier(LogisticRegression(C = c))
    mlb_clf.fit(X_train, y_train)
    return mlb_clf


def tfidf_features(X_train, X_test_1,X_test_2,X_test_3,X_test_4):
    
        tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df = 0.9, min_df = 5, token_pattern = '(\S+)')
        X_train = tfidf_vectorizer.fit_transform(X_train.values.astype('U'))
        X_test_1 = tfidf_vectorizer.transform(X_test_1.values.astype('U'))
        X_test_2 = tfidf_vectorizer.transform(X_test_2.values.astype('U'))
        X_test_3 = tfidf_vectorizer.transform(X_test_3.values.astype('U'))
        X_test_4 = tfidf_vectorizer.transform(X_test_4.values.astype('U'))
    
        return X_train, X_test_1, X_test_2 , X_test_3, X_test_4 , tfidf_vectorizer.vocabulary_



y_train=df1['tags']
X_train=df1['title']
X_test_1=df_test_1
X_test_2=df_test_2
X_test_3=df_test_3
X_test_4=df_test_4

from collections import defaultdict
tags_counts= defaultdict(int)
#words_counts_title= defaultdict(int)
for labels in y_train:
    for tag in labels:
        tags_counts[tag] += 1
#print(most_common_tags)    
    
from sklearn.feature_extraction.text import TfidfVectorizer

X_train_tfidf, X_test_1_tfidf ,  X_test_2_tfidf ,X_test_3_tfidf,X_test_4_tfidf,tfidf_vocab = tfidf_features(X_train, X_test_1,X_test_2,X_test_3,X_test_4)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}
    # Check if c++ or c# are in our vocabulary, as they are obviously important tokens in our tags prediction task.
#tfidf_vocab['c#'], tfidf_vocab['c++']
print('X_train_tfidf shape', X_train_tfidf.shape)

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
"""l=list(most_common_tags.keys())
#mlb = MultiLabelBinarizer(classes=sorted(most_common_tags.keys()))
mlb = MultiLabelBinarizer(classes=l)"""
y_train_bin = mlb.fit_transform(y_train)

    
    
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier


X_train_tfidf shape (31346, 6093)


In [8]:
classifier_tfidf = train_classifier(X_train_tfidf, y_train_bin)

In [14]:
#running X_test_1
final__predicted_labels_tfidf = classifier_tfidf.predict(X_test_4_tfidf)
y_val_pred_inversed = mlb.inverse_transform(final__predicted_labels_tfidf)


for i in range(len(df_test_4)):
    x=('{}'.format('|'.join(y_val_pred_inversed[i])))
    ans.append(x)

In [15]:
print(len(ans))

194323


In [16]:
ans_df= pd.DataFrame({'tags':ans})

In [17]:
ans_df['id']=df_t['id']
#ans_df['id']=df_t['id'][150000:194323]
ans_df=ans_df[['id','tags']]
ans_df.head(100)

,id,tags
0,HE-efbc27d,
1,HE-d1fd267,
2,HE-ffd4152,jquery
3,HE-d3ab268,
4,HE-ed2fa45,
5,HE-d49e9e8,
6,HE-eb48a78,
7,HE-ea1d270,silverlight
8,HE-c9b93a9,
9,HE-f575f7f,android


In [18]:
with open('420000_451346_submit.csv', 'a') as f:
    ans_df.to_csv(f, header=False)

In [117]:
#df_t['id'][50000:50002]